# How To Use NLP# 텍스트 분석과 챗봇 만들기

## 3. 베이즈 정리로 텍스트 분류하기

### 3.1 Text Classifier

In [1]:
   def fit(text, spam):
        """ 텍스트 학습 """
        word_list = text.split()
        for word in word_list:
            word_count(word, spam)
        spam_calc(spam)        

In [2]:
   def word_count(word, spam):
        # 단어를 스팸 카테고리에 추가하기
        if not spam in word_dict:
            word_dict[spam] = {}
        if not word in word_dict[spam]:
            word_dict[spam][word] = 0
        word_dict[spam][word] += 1
        words.add(word)
    

In [3]:
     def spam_calc(spam):
        if not spam in spam_dict:
            spam_dict[spam] = 0
        spam_dict[spam] += 1

In [4]:
words=set()
word_dict={}
spam_dict={}

fit('파격세일 - 오늘까지만 30% 할인', 'spam')
fit("쿠폰 선물 & 무료 배송", "spam")
fit("오늘 일정 확인", "ham")

In [5]:
print(word_dict)
print(spam_dict)

{'spam': {'파격세일': 1, '-': 1, '오늘까지만': 1, '30%': 1, '할인': 1, '쿠폰': 1, '선물': 1, '&': 1, '무료': 1, '배송': 1}, 'ham': {'오늘': 1, '일정': 1, '확인': 1}}
{'spam': 2, 'ham': 1}


In [6]:
import math, sys
from konlpy.tag import Twitter
class BayesianClassifier:

    """ 베이지안 분류기
        세부기능 1 / 2로 구분
    """
    def __init__(self):
        self.words = set() # 출현   단어 기록
        self.word_dict = {} # 단어 출현 횟수 기록 , dict형
        self.spam_dict = {} # 스팸 출현 횟수 기록 , dict형
        
   # 세부기능 1: 텍스트 전체서 스팸 계산하기
    def fit(self, text, spam):
        """ 텍스트 학습 """
        word_list = self.split(text)
        for word in word_list:
            self.word_count(word, spam)
        self.spam_calc(spam)        
    
    # 세부기능 1: 형태소 분석하기 
    def split(self, text):
        results = []
        twitter = Twitter()
        # 단어의 기본형 사용
        postag = twitter.pos(text, norm=True, stem=True)
        for word in postag:
            # 어미/조사/구두점 등은 대상에서 제외 
            if not word[1] in ["Josa", "Eomi", "Punctuation"]:
                results.append(word[0])
        return results
    
    # 세부기능 1: 단어와 스팸의 출현 횟수 추가 단어 추가
    def word_count(self, word, spam):
        # 단어를 스팸 카테고리에 추가하기
        if not spam in self.word_dict: #초기화 1. word_dict에 spam카테고리가 없으면 만들어준다
            self.word_dict[spam] = {}
        if not word in self.word_dict[spam]: #초기화 2. spam카테고리에 아무것도 없으면 단어 개수를 넣고 단어를 추가한다. 
            self.word_dict[spam][word] = 0
        self.word_dict[spam][word] += 1 #스팸 단어 카운트 증가
        self.words.add(word) #단어 추가
    
    # 세부기능 1: 스팸 계산하기
    def spam_calc(self, spam):
        if not spam in self.spam_dict:
            self.spam_dict[spam] = 0
        self.spam_dict[spam] += 1
        

   # 세부기능 2: 예측하기 
    def predict(self, text):
        spam_select = None
        max_score = -sys.maxsize 
        
        words = self.split(text)
        score_list = []
        for spam in self.spam_dict.keys():
            score = self.score(words, spam)
            score_list.append((spam, score)) 
            if score > max_score:
                max_score = score
                spam_select = spam
        return spam_select, score_list
    
    # P(B)*P(A|B)= 단어 리스트에 점수 매기기
    def score(self, words, spam):
        score = math.log(self.spam_prob(spam)) #Tips. 확률이므로 log값을 사용해 downflow예방
        for word in words:
            score += math.log(self.word_prob(word, spam)) 
        return score
        
    # 스팸 내부의 단어 출현 횟수 구하기
    def get_word_count(self, word, spam):
        if word in self.word_dict[spam]:
            return self.word_dict[spam][word]
        else:
            return 0
    
    # P(B)= 전체 문서에서 해당 카테고리로 분류될 확률 계산
    def spam_prob(self, spam):
        sum_spam = sum(self.spam_dict.values()) #전체 문장의 수
        spam_v = self.spam_dict[spam] #새로 들어가는 문장의 class
        return spam_v / sum_spam
        
    # P(A|B)= 스팸 내부의 단어 출현율 계산
    def word_prob(self, word, spam):
        n = self.get_word_count(word, spam) + 1 #학습 사전에 없는 단어가 나오면 spam확률이 0이 되므로
        d = sum(self.word_dict[spam].values()) + len(self.words)
        return n / d

### 3.2 Baysian Classifier 

In [7]:
bc = BayesianClassifier()
# 텍스트 학습
bc.fit("파격 세일 - 오늘까지만 30% 할인", "광고")
bc.fit("쿠폰 선물 & 무료 배송", "광고")
bc.fit("현데계 백화점 세일", "광고")
bc.fit("봄과 함께 찾아온 따뜻한 신제품 소식", "광고")
bc.fit("인기 제품 기간 한정 세일", "광고")
bc.fit("오늘 일정 확인", "중요")
bc.fit("프로젝트 진행 상황 보고","중요")
bc.fit("계약 잘 부탁드립니다","중요")
bc.fit("회의 일정이 등록되었습니다.","중요")
bc.fit("오늘 일정이 없습니다.","중요")
# 예측
pre, scorelist = bc.predict("재고 정리 할인, 무료 배송")
print("결과 =", pre)
print(scorelist)

결과 = 광고
[('광고', -19.00139285840871), ('중요', -20.449365773467083)]


## Summerize
**베이즈 정리를 활용해 스팸 필터링 등 classification이 가능하다**  
**단어/카테고리 정리만 가지고 classification이 가능하다**